# Nova seção

In [ ]:
pip install pyspark

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:436)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecution(ChauffeurState.scala:1266)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:983)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:573)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:669)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:687)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:426)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:216)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:424)
	at com.databricks.logging.Usag

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types  as T
from datetime import datetime, timedelta
from pyspark.sql.types import DecimalType
import re
from pyspark.sql.window import Window

In [ ]:
spark = (
    SparkSession
    .builder.appName("test_NAVA")
    .config('spark.sql.repl.eagerEval.enabled', True)
    .master("local[*]").getOrCreate()
)

In [ ]:
df_ini=spark.read.csv('/content/tabela_saldo_inicial.txt', sep=';',header=True)
df_mov1=spark.read.csv('/content/movimentacao_dia_02_04_2022.txt', sep=';',header=True)
df_mov2=spark.read.csv('/content/movimentacao_dia_03_04_2022.txt', sep=';',header=True)

In [ ]:
display(df_ini)

Nome,CPF,Saldo_Dia,data
Maria,00000000001,523.86,2022-04-01
José,00000000002,917.78,2022-04-01
João,00000000003,321.84,2022-04-01
Paulo,00000000004,271.51,2022-04-01
Pedro,00000000005,225.55,2022-04-01
Antonio,00000000006,875.50,2022-04-01
Marcos,00000000007,365.88,2022-04-01
Luiz,00000000008,832.63,2022-04-01
Arthur,00000000009,221.12,2022-04-01


In [ ]:
display(df_mov1)

Nome,CPF,Movimentacao_dia,data
Paulo,00000000004,-32.56,02/04/2022
Pedro,00000000005,-169.51,02/04/2022
Antonio,00000000006,444.99,02/04/2022
Marcos,00000000007,291.73,02/04/2022
Marcos,00000000007,-133.67,02/04/2022
Marcos,00000000007,134.8,02/04/2022
Pedro,00000000005,-108.42,02/04/2022
Pedro,00000000005,134.8,02/04/2022
Pedro,00000000005,230.76,02/04/2022
Paulo,00000000004,274.18,02/04/2022


In [ ]:
display(df_total)

Nome,CPF,Saldo_Dia,data
Maria,00000000001,523.86,2022-04-01
José,00000000002,917.78,2022-04-01
João,00000000003,321.84,2022-04-01
Paulo,00000000004,271.51,2022-04-01
Pedro,00000000005,225.55,2022-04-01
Antonio,00000000006,875.50,2022-04-01
Marcos,00000000007,365.88,2022-04-01
Luiz,00000000008,832.63,2022-04-01
Arthur,00000000009,221.12,2022-04-01
Paulo,00000000004,-32.56,2022-04-02


In [ ]:
df_mov_total = df_mov1.union(df_mov2)

In [ ]:
df_mov_total = (df_mov_total.withColumn('data', F.to_date(df_mov_total['data'], 'dd/MM/yyyy'))
                .withColumn("Movimentacao_dia",F.col("Movimentacao_dia").cast(DecimalType(18,2)))
                            .withColumnRenamed("Movimentacao_dia", "Saldo_Dia")
)
df_ini = (df_ini.withColumn('data', F.to_date(df_ini['data'], 'dd/MM/yyyy'))
          .withColumn("Saldo_Inicial_CC",F.col("Saldo_Inicial_CC").cast(DecimalType(18,2)))
          .withColumnRenamed("Saldo_Inicial_CC", "Saldo_Dia"))

df_total = df_ini.union(df_mov_total)

In [ ]:
# Calculando o saldo final usando janelas
window_spec = Window.partitionBy("CPF").orderBy("data")
df = df_total.withColumn("saldo_final", F.round(F.sum("Saldo_Dia").over(window_spec), 2))

display(df)

Nome,CPF,Saldo_Dia,data,saldo_final
Maria,00000000001,523.86,2022-04-01,523.86
Maria,00000000001,157.62,2022-04-03,1415.18
Maria,00000000001,518.89,2022-04-03,1415.18
Maria,00000000001,214.81,2022-04-03,1415.18
José,00000000002,917.78,2022-04-01,917.78
José,00000000002,56.80,2022-04-03,1466.71
José,00000000002,492.13,2022-04-03,1466.71
João,00000000003,321.84,2022-04-01,321.84
João,00000000003,-106.57,2022-04-03,12.57
João,00000000003,-406.67,2022-04-03,12.57


In [ ]:
df.write.csv('/content/movimentacao',sep=';',mode='overwrite', header=True, encoding='iso-8859-1')

In [ ]:
def list_extrato(l: list) -> list:
    idx = None
    for i in l:
        if re.search(r"Data: \d{2}/\d{2}/\d{4} \| Saldo: ", i):
            idx = i
    dt = idx.split(' |')[0].replace('Data:', '').strip()
    dt = datetime.strptime(dt, '%d/%m/%Y')
    vl = idx.split(' |')[1].replace('Saldo:', '').strip()
    while dt < datetime(2022, 4, 3):
        dt += timedelta(days=1)
        lista.append(f'Data: {dt.strftime("%d/%m/%Y")} | Saldo: {vl}')
    return l

def saldo_final(cpf_cliente: int) -> dict:
    saldo_final = {}
    saldo_inicial = 0
    # For para pegar o saldo final de cada dia
    for row_saldo in df_total.filter(df_total["CPF"] == cpf_cliente).rdd.toLocalIterator():
        saldo_inicial += row_saldo['Saldo_Dia']
        saldo_final[row_saldo['data'].strftime("%d/%m/%Y")] = saldo_inicial
    return saldo_final
base_clientes = df_total.dropDuplicates(subset=["CPF"])
if __name__ == "__main__":
    lista_final = []
    for row in base_clientes.rdd.toLocalIterator():
        data_anterior = row["data"]
        saldo = row["Saldo_Dia"]
        lista = [f'\n{row["Nome"]}', f'Data: {data_anterior.strftime("%d/%m/%Y")} | Saldo: {saldo}']
        saldo_final_dia = saldo_final(row['CPF'])
        saldo_final_dia[data_anterior.strftime("%d/%m/%Y")] = saldo
        for row_mov in df_total.filter(df_total["CPF"] == row['CPF']).rdd.toLocalIterator():
            if data_anterior == row_mov['data']:
                saldo += row_mov['Saldo_Dia']
                lista.append(f"\t\tMov: {row_mov['data'].strftime('%d/%m/%Y')} | {(row_mov['Saldo_Dia'])}")
            else:
                if(row_mov['data'] - data_anterior).days == 2:
                    dia = row_mov['data'] - timedelta(days=1)
                    valor  = row_mov['data'] - timedelta(days=2)
                    lista.append(f'Data: {dia.strftime("%d/%m/%Y")} | Saldo: XYZ{valor.strftime("%d/%m/%Y")}')
                saldo += row_mov['Saldo_Dia']
                lista.append(f"Data: {row_mov['data'].strftime('%d/%m/%Y')} | Saldo: XYZ{row_mov['data'].strftime('%d/%m/%Y')}")
                lista.append(f"\t\tMov: {row_mov['data'].strftime('%d/%m/%Y')} | {row_mov['Saldo_Dia']}")
            data_anterior = row_mov.data
        saldo_final_dia[data_anterior.strftime("%d/%m/%Y")] = saldo
        lista = [re.sub(r'XYZ(\d{2}/\d{2}/\d{4})', lambda match: str(saldo_final_dia.get(match.group(1), match.group(0))), linha) for linha in lista]
        lista = list_extrato(lista)

        for x in lista:
            lista_final.append(x)
            print(x)




Maria
Data: 01/04/2022 | Saldo: 523.86
		Mov: 01/04/2022 | 523.86
Data: 02/04/2022 | Saldo: 523.86
Data: 03/04/2022 | Saldo: 1939.04
		Mov: 03/04/2022 | 157.62
		Mov: 03/04/2022 | 518.89
		Mov: 03/04/2022 | 214.81

José
Data: 01/04/2022 | Saldo: 917.78
		Mov: 01/04/2022 | 917.78
Data: 02/04/2022 | Saldo: 917.78
Data: 03/04/2022 | Saldo: 2384.49
		Mov: 03/04/2022 | 56.80
		Mov: 03/04/2022 | 492.13

João
Data: 01/04/2022 | Saldo: 321.84
		Mov: 01/04/2022 | 321.84
Data: 02/04/2022 | Saldo: 321.84
Data: 03/04/2022 | Saldo: 334.41
		Mov: 03/04/2022 | -106.57
		Mov: 03/04/2022 | -406.67
		Mov: 03/04/2022 | 203.97

Paulo
Data: 01/04/2022 | Saldo: 271.51
		Mov: 01/04/2022 | 271.51
Data: 02/04/2022 | Saldo: 784.64
		Mov: 02/04/2022 | -32.56
		Mov: 02/04/2022 | 274.18
Data: 03/04/2022 | Saldo: 784.64

Pedro
Data: 01/04/2022 | Saldo: 225.55
		Mov: 01/04/2022 | 225.55
Data: 02/04/2022 | Saldo: 708.24
		Mov: 02/04/2022 | -169.51
		Mov: 02/04/2022 | -108.42
		Mov: 02/04/2022 | 134.80
		Mov: 02/04/2